In [ ]:
!nvidia-smi

Thu Feb 11 14:39:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install insightface
!pip install onnx
!pip install onnxruntime
!pip install mxnet-cu101
#mxnet-cu101 theo bản cuda

  Created wheel for insightface: filename=insightface-0.1.5-cp36-none-any.whl size=16766 sha256=2bcf81feef32e6f35e5e20b09e30d69eb1fd260ab8a6555f582387215a5e2aaa
  Stored in directory: /root/.cache/pip/wheels/11/b8/83/2bac12a833d66d58aa9705f5e7d94d2e9055f16f4698af0df7
Successfully built insightface
     |████████████████████████████████| 14.5MB 189kB/s 
     |████████████████████████████████| 4.1MB 13.7MB/s 
     |███████████████████████████████▌| 834.1MB 1.3MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x39884000 @  0x7face7a68615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.4MB 20kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10

In [ ]:
%cd\/content/drive/MyDrive/AI4E_DL/CW2  
#link to folder

/content/drive/MyDrive/AI4E_DL/CW2


In [ ]:
import insightface
from Align import get_reference_facial_points, warp_and_crop_face
import cv2
import numpy as np
import onnxruntime
from imutils import paths
import time

In [ ]:
#load label and embedding dataset
emb = np.load('embedding_25.npy')
with open(r'./label.txt',  encoding="utf8") as fobj:
    data = fobj.read()
name = data.split('\n')

In [ ]:
#load model
model_detect = insightface.model_zoo.get_model('retinaface_r50_v1')
model_detect.prepare(ctx_id = 0, nms=0.4)
reference = get_reference_facial_points(default_square=True)

model_arc = onnxruntime.InferenceSession(r'arcface_r100.onnx')
print("Load model: Done")

Model file is not found. Downloading.


100%|██████████| 107062/107062 [00:08<00:00, 12209.05KB/s]


[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True
Load model: Done


In [ ]:
#pre process image
def preprocess(frame):
    bbox, landmarks = model_detect.detect(frame, threshold=0.5, scale=1.0)
    landmarks = landmarks.astype('int')
    landmarks = landmarks[0].tolist()
    img = warp_and_crop_face(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), landmarks, reference,(112, 112))
    img_predict = np.transpose(img, (2, 0, 1))
    img_predict = np.expand_dims(img_predict, axis=0)
    img_predict = np.float32(img_predict)
    return img, img_predict

In [ ]:
def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [ ]:
def predict(link):
  img = cv2.imread(link)
  img = cv2.resize(img, (1000,1000))
  try:
    img_crop, img_predict = preprocess(img)
    embedding = model_arc.run(None, {'data': img_predict})[0][0]
      
    distance = []
    for i in range(emb.shape[0]):
      min_ = 2
      for j in range(emb.shape[1]):
        dist = findCosineDistance(embedding, emb[i,j])
        if min_ > dist:
          min_ = dist
      distance.append(min_)
    min_ = min(distance)
    if min_ <0.6:
      print('{} : {}'.format(link.split('/')[-1], name[distance.index(min_)]))
    else:
      print('{} : {}'.format(link.split('/')[-1], 'Unknown'))
  except:
    print('{} : Fail'.format(link))


In [ ]:
def main(link):
  imagePaths = list(paths.list_images(link))
  for i in imagePaths:
    predict(i)

In [ ]:

main(link)

Finish: 0.0s


# **TEST**

In [ ]:
def predict_(link):
  img = cv2.imread(link)
  img = cv2.resize(img, (1000,1000))
  true = 0 
  try:
    img_crop, img_predict = preprocess(img)
    embedding = model_arc.run(None, {'data': img_predict})[0][0]
      
    distance = []
    for i in range(emb.shape[0]):
      min_ = 2
      for j in range(emb.shape[1]):
        dist = findCosineDistance(embedding, emb[i,j])
        if min_ > dist:
          min_ = dist
      distance.append(min_)
    min_ = min(distance)
    if min_ <0.6:
      print('{} :, label: {}, predict: {}'.format(link.split('/')[-1], name[int(link.split('/')[-2])], name[distance.index(min_)]))
      true += 1
    else:
      print('{} : {}'.format(link.split('/')[-1], 'Unknown'))
    
  except:
    print('Fail')
  return true

In [ ]:
def main(link):
  imagePaths = list(paths.list_images(link))
  for i in imagePaths:
    predict_(i)

In [ ]:
true = main('/content/drive/MyDrive/AI4E_DL/CW2/data_test')

/content/drive/MyDrive/AI4E_DL/CW2/Align.py:87: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r, _, _, _ = lstsq(X, U)


30.png :, label: Thắng, predict: Linh
39.png :, label: Thắng, predict: Hiếu
6.png :, label: Thắng, predict: Thắng
27.png :, label: Thắng, predict: Thắng
21.png :, label: Thắng, predict: Thắng
42.png :, label: Thắng, predict: Thắng
3.png :, label: Thắng, predict: Thắng
24.png :, label: Thắng, predict: Thắng
0.png :, label: Thắng, predict: Thắng
18.png :, label: Thắng, predict: Thắng
15.png :, label: Thắng, predict: Thắng
33.png :, label: Thắng, predict: Thắng
9.png :, label: Thắng, predict: Thắng
12.png :, label: Thắng, predict: Thắng
36.png :, label: Thắng, predict: Thắng
90.png :, label: Thắng, predict: Thắng
66.png :, label: Thắng, predict: Thắng
51.png :, label: Thắng, predict: Thắng
63.png :, label: Thắng, predict: Thắng
84.png :, label: Thắng, predict: Thắng
96.png :, label: Thắng, predict: Thắng
75.png :, label: Thắng, predict: Thắng
60.png :, label: Thắng, predict: Thắng
78.png :, label: Thắng, predict: Thắng
54.png :, label: Thắng, predict: Thắng
45.png :, label: Thắng, predict